### К вопросу о том, как можно искать ошибки в тексте
**Расстояние Хэмминга** - число позиций, в которых соответствующие символы двух слов одинаковой длины различны (плюс насколько второе слово длиннее первого).

**Одноклассники**<br>
Аднаклассники  H=2<br>
Одоклассники    H=10<br>
Однокласссники H=5<br>

Для **Расстояние Левенштейна** вводится три вида ошибок: вставки (*ошиббка*), удаления (*ошика*) и замены (*ашибка*).<br>
Тогда оно определяется как минимальное количество ошибок, которое надо сделать в одном слове, чтобы получить второе.

**Одноклассники**<br>
Аднаклассники  L=2<br>
Одоклассники   L=1<br>
Однокласссники L=1

**Алгоритм Библиотеки Конгресса США** основывается на том, что читатель некорректно произносит название книги на языке, который оба не понимают.
- Заменяем буквы на похожие (б-п, о-а, ...).
- Заменяем повторяющиеся буквы на одно вхождение.
- Дополнительные действия.
- Ищем (по Левенштейну).

Напишем простую реализации алгоритма нахождения расстояния Левенштейна и замеряем её производительность.

In [1]:
def naiveLeven(str1, str2):
    if len(str1) == len(str2) and len(str1) == 0:
        return 0
    elif len(str1) * len(str2) == 0:
        return abs(len(str1) - len(str2))
    elif str1[0] == str2[0]:
        return naiveLeven(str1[1:], str2[1:])
    else:
        n0 = naiveLeven(str1[1:], str2[1:])
        n1 = naiveLeven(str1[1:], str2)
        n2 = naiveLeven(str1, str2[1:])
        return min(n0, n1, n2)+1

In [2]:
print(naiveLeven("мама", "мама"))
print(naiveLeven("мама", "пама"))
print(naiveLeven("мама", "папа"))
print(naiveLeven("мама", "маема"))
print(naiveLeven("мама", "ама"))
print(naiveLeven("мама", "паа"))


0
1
2
1
1
2


In [7]:
%%timeit
# naiveLeven("Комсомольск-на-Амуре", "Комсамольск-на-Омуре") # --
# naiveLeven("Клышинский", "Колтышинский") # 293
naiveLeven("Клышинский", "Клышинский") # 3.31

3.31 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Как видно, для строк большой длины поиск производится за слишком большое время.  
Усложним алгоритм, добавив к нему предельное количество ошибок, которое может быть совершено в слове. Помимо этого, придется передавать количество ошибок, которые мы уже нашли при сравнении текущей пары слов.

In [8]:
def naiveLeven2(str1, str2, cerr, thr):
    if cerr > thr:
        return 1000000
    if len(str1) == len(str2) and len(str1) == 0:
        return 0
    elif len(str1) * len(str2) == 0:
        return abs(len(str1) - len(str2))
    elif str1[0] == str2[0]:
        return naiveLeven2(str1[1:], str2[1:], cerr, thr)
    else:
        n0 = naiveLeven2(str1[1:], str2[1:], cerr+1, thr)
        n1 = naiveLeven2(str1[1:], str2, cerr+1, thr)
        n2 = naiveLeven2(str1, str2[1:], cerr+1, thr)
        return min(n0, n1, n2)+1

In [11]:
%%timeit
# naiveLeven2("Комсомольск-на-Амуре", "Комсамольск-на-Омуре", 0, 3) # 52
# naiveLeven2("Клышинский", "Колтышинский", 0, 3) # 36
naiveLeven2("Клышинский", "Клышинский", 0, 3) # 3.5

3.5 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Стало, безусловно, быстрее. Но мы понимаем где тратится больше всего времени - если буквы не совпали, мы должны проверить несколько гипотез. При том может выясниться, что на первом шаге было найдено лучшее решение и дальнейшая проверка может не проводиться. Поэтому будем менять пороговое значение в следующих шагав, сообщив алгоритму, что не надо допускать больше ошибок, чем для уже найденного решения.

In [12]:
def Leven(str1, str2, cerr, thr):
    if cerr > thr:
        return 1000000
    if len(str1) == len(str2) and len(str1) == 0:
        return 0
    elif len(str1) * len(str2) == 0:
        return abs(len(str1) - len(str2))
    elif str1[0] == str2[0]:
        return Leven(str1[1:], str2[1:], cerr, thr)
    else:
        n0 = Leven(str1[1:], str2[1:], cerr+1, thr)
        n1 = Leven(str1[1:], str2, cerr+1, min(thr, cerr+n0))
        n2 = Leven(str1, str2[1:], cerr+1, min(thr, cerr+min(n0, n1)))
        return min(n0, n1, n2)+1

In [15]:
%%timeit
# Leven("Комсомольск-на-Амуре", "Комсамольск-на-Омуре", 0, 3) # 10.2
# Leven("Клышинский", "Колтышинский", 0, 3) # 26.9
Leven("Клышинский", "Клышинский", 0, 3) # 3.38

3.38 µs ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Но ещё быстрее алгоритм будет работать [Алгоритм Вагнера — Фишера](https://habr.com/ru/articles/676858) или его модификация - алгоритм [Хиршберга](https://habr.com/ru/articles/117063/). Первому требуется O(m*n) памяти, второй линеен по количеству памяти.

Эти и многие другие алгоритмы реализованы в библиотеке `textdistance` - https://pypi.org/project/textdistance/ .

In [17]:
!pip3 install textdistance
!pip3 install "textdistance[extras]"

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 779.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.7 MB/s eta 0:00:0000:0100:01
  Created wheel for pyxDamerauLevenshtein: filename=pyxDamerauLevenshtein-1.7.1-cp310-cp310-linux_x86_64.whl size=70672 sha256=f0bed58110957b6406dca8c8dc2f86d3777a70aec9626b46475a126cd6ede9c3
  Stored in directory: /home/edward/.cache/pip/wheels/a7/6f/29/3c4490c70e8dde5a58752c4e63aaf46df9d6a24861d8a06c08
Successfully built pyxDamerauLevenshtein


In [18]:
import textdistance


In [19]:
textdistance.hamming('test', 'text')

1

In [22]:
%%timeit
textdistance.levenshtein("Комсомольск-на-Амуре", "Комсамольск-на-Омуре") # 2.12
# textdistance.levenshtein("Клышинский", "Колтышинский") # 2.27
# textdistance.levenshtein("Клышинский", "Клышинский") # 0.59

2.12 µs ± 46.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Наконец, библиотека для поиска ошибок.

In [27]:
!pip3 install -U language_tool_python

Defaulting to user installation because normal site-packages is not writeable


In [28]:
import language_tool_python

In [29]:
tool = language_tool_python.LanguageTool('ru')

Unzipping /tmp/tmplxrg3ak3.zip to /home/edward/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /home/edward/.cache/language_tool_python.


In [31]:
with open("data/lenta2018.txt", encoding="utf-8") as news_file: # Файл с новостями.
    lenta_news = [n.split("-----\n")[1] for n in news_file.read().split("=====\n")[1:]]

In [32]:
%%time
for news in lenta_news[:100]:
    matches = tool.check(news)
    if len(matches) > 0:
        print(matches)

[Match({'ruleId': 'MORFOLOGIK_RULE_RU_RU', 'message': 'Возможно найдена орфографическая ошибка.', 'replacements': [], 'offsetInContext': 43, 'context': '...краине имеется, видимо, полный комплект производственно-конструкторской документации на эту ракету, как и ведет...', 'offset': 476, 'errorLength': 31, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': 'Об этом пишет военный блог bmpd, который ведут сотрудники московского Центра анализа стратегий и технологий.«Напомним, что в советский период серийное производство ракет 3М24 (Х-35) планировалось организовать на Харьковском авиационном заводе (нынешнее ХГАПП), так что на Украине имеется, видимо, полный комплект производственно-конструкторской документации на эту ракету, как и ведется производство ее двигателя», — отметили эксперты.'}), Match({'ruleId': 'SENTENCE_WHITESPACE', 'message': 'Добавьте пробел между предложениями.', 'replacements': [' Разработчиком'], 'offsetInContext': 43, 'context': '...ство ее двигателя», — 